<a href="https://colab.research.google.com/drive/1l303DcxP66J8i_mNxSKRPirU2HXXq5OB?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multilingual Search with Cohere & Pinecone

In [3]:
! pip install cohere annoy pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 21.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.0/211.0 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00
  Created wheel for annoy: filename=annoy-1.17.3-cp310-cp310-linux_x86_64.whl size=552453 sha256=7d8bf551667a54c2f5dd0e336443d5db9fcf24d7a5e164b4144dbbfd4a919fd2
  Stored in directory: /root/.cache/pip/wheels/64/8a/da/f714bcf46c5efdcfcac0559e63370c21abe961c48e3992465a
Successfully built annoy
  Attempting uninstall: importlib_metadata
    Found existing installation: importlib-metadata 7.0

In [4]:
import cohere
import os
import random
import pandas as pd
from annoy import AnnoyIndex
import numpy as np

In [5]:
api_key = "" # add your cohere API key here
co = cohere.Client(api_key)

# Quranic: Given a question, find the answer from the Quran.

### Import a list of documents

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
raw_df = pd.read_csv("/content/gdrive/MyDrive/data/quran.csv", index_col=0)
print(raw_df.shape)
hq_df = raw_df
hq_df.head()

(6236, 8)


,Surah,Ayat,Arabic,Translation1,Translation2,Translation3,Tafaseer1,Tafaseer2
Name,,,,,,,,
The Opening,1,1,بِسمِ ٱلله الرَّحْمٰنِ الرَّحِيـمِ,"All praise be to Allah alone, the Sustainer of...","In the Name of God, the Merciful, the Compassi...","In the name of Allah, the Beneficent, the Merc...",In the Name of God the Compassionate the Merciful,"In the name of Allah, the Beneficent, the Merc..."
The Opening,1,2,ٱلْحَمْدُ للَّهِ رَبِّ ٱلْعَالَمِينَ,"Most Compassionate, Ever-Merciful,","Praise belongs to God, the Lord of all Being,","Praise be to Allah, Lord of the Worlds,",In the Name of God the name of a thing is that...,And on his authority it is related that Ibn 'A...
The Opening,1,3,ٱلرَّحْمـٰنِ ٱلرَّحِيمِ,Master of the Day of Judgment.,"the All-merciful, the All-compassionate,","The Beneficent, the Merciful.",The Compassionate the Merciful that is to say ...,(The Beneficent) the Gentle. (The Merciful) th...
The Opening,1,4,مَـٰلِكِ يَوْمِ ٱلدِّينِ,(O Allah!) You alone do we worship and to You ...,the Master of the Day of Doom.,"Owner of the Day of Judgment,",Master of the Day of Judgement that is the day...,(Owner of the Day of Judgement) the Arbitrator...
The Opening,1,5,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ,"Show us the straight path,",Thee only we serve; to Thee alone we pray for ...,Thee (alone) we worship; Thee (alone) we ask f...,You alone we worship and You alone we ask for ...,"(Thee (alone) we worship), we turn to you as t..."


In [8]:
df = hq_df.drop(columns=['Translation2','Translation3','Tafaseer2'])

In [9]:
df.shape

(6236, 5)

In [10]:
# Print a few sample documents
texts = df["Arabic"].tolist()

random.seed(100)
for item in random.sample(texts, 5):
  print(item)

وَمَا لَهُمْ أَلاَّ يُعَذِّبَهُمُ ٱللَّهُ وَهُمْ يَصُدُّونَ عَنِ ٱلْمَسْجِدِ ٱلْحَرَامِ وَمَا كَانُوۤاْ أَوْلِيَآءَهُ إِنْ أَوْلِيَآؤُهُ إِلاَّ ٱلْمُتَّقُونَ وَلَـٰكِنَّ أَكْثَرَهُمْ لاَ يَعْلَمُونَ
أَلَمْ أَعْهَدْ إِلَيْكُمْ يٰبَنِيۤ ءَادَمَ أَن لاَّ تَعْبُدُواْ ٱلشَّيطَانَ إِنَّهُ لَكُمْ عَدُوٌّ مُّبِينٌ
وَمَا لِيَ لاَ أَعْبُدُ ٱلَّذِي فَطَرَنِي وَإِلَيْهِ تُرْجَعُونَ
قَالُواْ ٱتَّخَذَ ٱللَّهُ وَلَداً سُبْحَانَهُ هُوَ ٱلْغَنِيُّ لَهُ مَا فِي ٱلسَّمَٰوَٰت وَمَا فِي ٱلأَرْضِ إِنْ عِندَكُمْ مِّن سُلْطَانٍ بِهَـٰذَآ أَتقُولُونَ عَلَى ٱللَّهِ مَا لاَ تَعْلَمُونَ
ثُمَّ إِذَا شَآءَ أَنشَرَهُ


### Embed the documents and store them in an index

In [11]:
response = co.embed(texts=texts, model='multilingual-22-12').embeddings
embeds = np.array(response)
# Embed the documents and store in index
search_index = AnnoyIndex(embeds.shape[1], 'angular')
# Add all the vectors to the search index
for i in range(len(embeds)):
    search_index.add_item(i, embeds[i])

search_index.build(100) # 10 trees
search_index.save('quran_index.ann')

True

In [12]:
embeds.shape

(6236, 768)

### Enter a query

In [13]:
# They are all the same ingredients
# Expected: Output recipes are the same
queries = ["مَـٰلِكِ يَوْمِ ٱلدِّينِ", "What does the quran say about the dis-believers"]

queries_lang = ["Arabic", "English"]

### Return the results most similar to the query



In [14]:
results_list = []

for idx, q in enumerate(queries):

    # Retrieve the nearest neighbors
    query_embed = co.embed(texts=[q], model='multilingual-22-12').embeddings
    similar_item_ids, _ = search_index.get_nns_by_vector(query_embed[0], 5, include_distances=True)

    if len(similar_item_ids) >= 2:
        # Format the results
        results = pd.DataFrame(data={'Surah': df.iloc[similar_item_ids[0]]['Surah'],
                                     'Ayah': df.iloc[similar_item_ids[0]]['Ayat'],
                                     'Arabic': df.iloc[similar_item_ids[0]]['Arabic'],
                                     'Translation': df.iloc[similar_item_ids[0]]['Translation1']}, index=[idx])

        results_list.append(results)
        print(f"Query:'{q}'\nNearest neighbors:")
        print(queries_lang[idx])
        print(results)
        print("\n")
    else:
        print("Not enough similar items found for query:", q)

Query:'مَـٰلِكِ يَوْمِ ٱلدِّينِ'
Nearest neighbors:
Arabic
   Surah  Ayah                    Arabic  \
0      1     4  مَـٰلِكِ يَوْمِ ٱلدِّينِ   

                                         Translation  
0  (O Allah!) You alone do we worship and to You ...  


Query:'What does the quran say about the dis-believers'
Nearest neighbors:
English
   Surah  Ayah                                             Arabic  \
1      9    45  إِنَّمَا يَسْتَأْذِنُكَ ٱلَّذِينَ لاَ يُؤْمِنُ...   

                                         Translation  
1  Only those who do not believe in Allah and the...  




### Initialise the pinecone index


In [15]:
from pinecone import Pinecone

pc = Pinecone(api_key="") # add your pinecone API key here

index_name = 'quranic'

# if the index does not exist, we create it
if index_name not in pc.list_indexes().names():
    pc.create_index(
        index_name,
        dimension=shape[1],
        metric='cosine'
    )

# connect to index
index = pc.Index(index_name)

In [16]:
import numpy as np

shape = np.array(embeds).shape
shape

(6236, 768)

### Upsert the data into pinecone


In [19]:
batch_size = 128

ids = [str(i) for i in range(shape[0])]
# create list of metadata dictionaries
meta = [{'Arabic': arabic, 'Surah': surah, 'Ayat': ayat, 'Translation1': translation, 'Tafaseer1': tafaseer} for arabic, surah, ayat, translation, tafaseer in zip(df['Arabic'], df['Surah'], df['Ayat'], df['Translation1'], df['Tafaseer1'])]

# create list of (id, vector, metadata) tuples to be upserted
to_upsert = list(zip(ids, embeds, meta))

for i in range(0, shape[0], batch_size):
    i_end = min(i+batch_size, shape[0])
    index.upsert(vectors=to_upsert[i:i_end])

# let's view the index statistics
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.06236,
 'namespaces': {'': {'vector_count': 6236}},
 'total_vector_count': 6236}

### Return the results most similar to the query from pinecone


In [20]:
query = "What does the quran say about the dis-believers?"

# create the query embedding
xq = co.embed(
    texts=[query],
    model='multilingual-22-12',
    truncate='NONE'
).embeddings

print(np.array(xq).shape)

# query, returning the top 10 most similar results
res = index.query(vector=xq, top_k=10, include_metadata=True)
res

(1, 768)


{'matches': [{'id': '1279',
              'metadata': {'Arabic': 'إِنَّمَا يَسْتَأْذِنُكَ ٱلَّذِينَ لاَ '
                                     'يُؤْمِنُونَ بِٱللَّهِ وَٱلْيَوْمِ '
                                     'ٱلآخِرِ وَٱرْتَابَتْ قُلُوبُهُمْ فَهُمْ '
                                     'فِي رَيْبِهِمْ يَتَرَدَّدُونَ',
                           'Ayat': 45.0,
                           'Surah': 9.0,
                           'Tafaseer1': 'They alone ask leave of you to stay '
                                        'behind who do not believe in God and '
                                        'the Last Day and whose hearts are '
                                        'doubtful uncertain about religion so '
                                        'in their doubt they waver they are '
                                        'confused.',
                           'Translation1': 'Only those who do not believe in '
                                           'Allah and the Last

In [24]:
for match in res['matches']:
    print(f"{match['score']:.2f}: {match['metadata']['Arabic']}")
    print(f"{match['score']:.2f}: {match['metadata']['Translation1']}")
    print(f"{match['score']:.2f}: {match['metadata']['Tafaseer1']}")

0.91: إِنَّمَا يَسْتَأْذِنُكَ ٱلَّذِينَ لاَ يُؤْمِنُونَ بِٱللَّهِ وَٱلْيَوْمِ ٱلآخِرِ وَٱرْتَابَتْ قُلُوبُهُمْ فَهُمْ فِي رَيْبِهِمْ يَتَرَدَّدُونَ
0.91: Only those who do not believe in Allah and the Last Day and whose hearts are in doubt will seek leave (i.e., exemption from fighting) from you. So in their doubt they are bewildered, lost in disorientation.
0.91: They alone ask leave of you to stay behind who do not believe in God and the Last Day and whose hearts are doubtful uncertain about religion so in their doubt they waver they are confused.
0.91: وَمَاذَا عَلَيْهِمْ لَوْ آمَنُواْ بِٱللَّهِ وَٱلْيَوْمِ ٱلآخِرِ وَأَنْفَقُواْ مِمَّا رَزَقَهُمُ ٱللَّهُ وَكَانَ ٱللَّهُ بِهِم عَلِيماً
0.91: And what loss would they have suffered, if they had believed in Allah and the Last Day, and spent (in His way) of what Allah had given them? And Allah is Well Aware of them (i.e., their state of affairs).
0.91: And what burden is on them if they were to believe in God and the Last Day and expend 